# Task 2: Analyze Your Data – How Well Do the Neurons Tell Orientation?
Petra Reuwsaat Paul

In [48]:
import numpy as np
import pandas as pd

In [49]:
# Load data from Task 1
data = np.load('result.npz')
result = data['result']
neurons_preferred = data['neurons']
stimulus_orientations = data['stimulus'] 

# Check dimensions
n_neurons, n_stimuli, n_trials = result.shape
print(f"Shape: {n_neurons} neurons, {n_stimuli} orientations, {n_trials} trials")

Shape: 50 neurons, 8 orientations, 30 trials


## Part 1: Decode the Orientation
Now that you’ve simulated how your 50 “neurons” fire when shown lines at different orientations, let’s put on the researcher's hat, and figure out how much information the "neurons" activity gives about the stimulus.
1. “Decode” the Orientation
● Use the neurons’ firing rates to estimate which orientation was shown, for each trial. (8 orientations, 30 trials, 240 in total).
● Use pandas to organize your data: if you have your data in a 3D numpy array (neurons × orientations × trials) you may want to reshape it into a “long” pandas DataFrame, with the columns like: neuron_id, stimulus_orientation, trial_number, firing_rate, and neuron’s preferred_angle.
● For each trial, use the firing rates of all neurons to estimate ("decode") the stimulus orientation that was shown on that trial.
(If you’re unsure how to proceed, see (*) at the bottom of the document)

### 1.1 Organize data into DataFrame

In [50]:
# Reshape 3D array into long format DataFrame
# Following assignment instructions for columns
data_list = []

for neuron_id in range(n_neurons):
    for stim_idx in range(n_stimuli):
        for trial_idx in range(n_trials):
            data_list.append({
                'neuron_id': neuron_id,
                'stimulus_orientation': stimulus_orientations[stim_idx],
                'trial_number': trial_idx,
                'firing_rate': result[neuron_id, stim_idx, trial_idx],
                'preferred_angle': neurons_preferred[neuron_id]
            })

df = pd.DataFrame(data_list)
print(f"DataFrame shape: {df.shape}")
df.head()

DataFrame shape: (12000, 5)


,neuron_id,stimulus_orientation,trial_number,firing_rate,preferred_angle
0,0,0.0,0,88.0,144.486885
1,0,0.0,1,80.0,144.486885
2,0,0.0,2,82.0,144.486885
3,0,0.0,3,79.0,144.486885
4,0,0.0,4,69.0,144.486885


In [51]:
# Quick statistics check
df.describe()

,neuron_id,stimulus_orientation,trial_number,firing_rate,preferred_angle
count,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000
mean,24.500000,78.750000,14.500000,63.557750,99.601817
std,14.431471,51.556125,8.655802,31.950804,50.815153
min,0.000000,0.000000,0.000000,0.000000,1.985328
25%,12.000000,39.375000,7.000000,38.000000,54.156412
50%,24.500000,78.750000,14.500000,70.000000,107.239902
75%,37.000000,118.125000,22.000000,90.000000,144.486885
max,49.000000,157.500000,29.000000,137.000000,179.533171


### 1.2 Create decoding function

Using population vector method from assignment hint

In [52]:
def decode_orientation(firing_rates, preferred_angles):
    # Step 1: Double angles and convert to radians
    # Orientation wraps every 180 degrees so we double it
    angles_rad = np.radians(preferred_angles * 2)

    # Step 2: Calculate weighted sum for x and y components
    # Each neuron votes with its firing rate
    x_component = np.sum(firing_rates * np.cos(angles_rad))
    y_component = np.sum(firing_rates * np.sin(angles_rad))

    # Step 3: Get angle using arctan2
    decoded_rad = np.arctan2(y_component, x_component)

    # Step 4: Convert back to degrees and divide by 2
    decoded_deg = np.degrees(decoded_rad) / 2

    # Make sure angle is between 0 and 180
    if decoded_deg < 0:
        decoded_deg += 180

    return decoded_deg

### 1.3 Decode all trials

In [53]:
# Decode orientation for each trial
decoded_list = []

for stim_idx in range(n_stimuli):
    for trial_idx in range(n_trials):
        # Get firing rates for all neurons in this trial
        firing_rates = result[:, stim_idx, trial_idx]

        # Decode using population vector
        decoded = decode_orientation(firing_rates, neurons_preferred)
        decoded_list.append(decoded)

# Create DataFrame with results
df_decoded = pd.DataFrame({
    'stimulus_orientation': np.repeat(stimulus_orientations, n_trials),
    'trial_number': np.tile(range(n_trials), n_stimuli),
    'decoded_orientation': decoded_list
})

print(f"Decoded {len(decoded_list)} trials")
df_decoded.head(10)

Decoded 240 trials


,stimulus_orientation,trial_number,decoded_orientation
0,0.0,0,129.321121
1,0.0,1,121.013715
2,0.0,2,114.287148
3,0.0,3,118.532626
4,0.0,4,131.845954
5,0.0,5,110.140919
6,0.0,6,122.532849
7,0.0,7,128.255358
8,0.0,8,118.203443
9,0.0,9,119.459868


## Part 2: Estimate Your Calculation
● Compare your decoded orientation to the true orientation shown in each trial.
● Calculate the absolute error.
● Compute the average error over all trials.

In [54]:
# Calculate absolute error
diff = np.abs(df_decoded['stimulus_orientation'] - df_decoded['decoded_orientation'])
df_decoded['absolute_error'] = np.minimum(diff, 180 - diff)

# Compute average error
mean_error = df_decoded['absolute_error'].mean()
std_error = df_decoded['absolute_error'].std()

print(f"Mean Absolute Error: {mean_error:.2f} degrees")
print(f"Standard Deviation: {std_error:.2f} degrees")

Mean Absolute Error: 53.03 degrees
Standard Deviation: 26.89 degrees


In [55]:
# Show examples of decoded vs true orientations
print("First 10 trials:")
print(df_decoded[['stimulus_orientation', 'decoded_orientation', 'absolute_error']].head(10))

First 10 trials:
   stimulus_orientation  decoded_orientation  absolute_error
0                   0.0           129.321121       50.678879
1                   0.0           121.013715       58.986285
2                   0.0           114.287148       65.712852
3                   0.0           118.532626       61.467374
4                   0.0           131.845954       48.154046
5                   0.0           110.140919       69.859081
6                   0.0           122.532849       57.467151
7                   0.0           128.255358       51.744642
8                   0.0           118.203443       61.796557
9                   0.0           119.459868       60.540132


In [56]:
# Error by stimulus orientation
print("\nAverage error for each orientation:")
error_by_orientation = df_decoded.groupby('stimulus_orientation')['absolute_error'].mean()
print(error_by_orientation)


Average error for each orientation:
stimulus_orientation
0.0      60.376009
22.5     75.894889
45.0     85.064846
67.5     80.768055
90.0     55.490753
112.5    27.099855
135.0     6.647434
157.5    32.936152
Name: absolute_error, dtype: float64


## Part 3: Critical Thinking
● How does decoding accuracy change when you use only the top 10 most responsive neurons vs. all 50 neurons?
● What happens to your decoding accuracy if you double the noise level (multiply Poisson parameter by 2)?

### 3.1 Top 10 neurons vs all 50 neurons

In [57]:
# Find the 10 most responsive neurons
# Using mean firing rate across all stimuli and trials
mean_firing_rates = result.mean(axis=(1, 2))
top_10_neurons = np.argsort(mean_firing_rates)[-10:]

print(f"Top 10 most responsive neurons: {top_10_neurons}")
print(f"Their mean firing rates: {mean_firing_rates[top_10_neurons]}")

Top 10 most responsive neurons: [29  7  8 17 15 27 33 12 28 45]
Their mean firing rates: [64.00833333 64.02083333 64.1        64.1125     64.1875     64.40833333
 64.46666667 64.58333333 64.90416667 65.11666667]


In [58]:
# Decode using only top 10 neurons
decoded_top10 = []

for stim_idx in range(n_stimuli):
    for trial_idx in range(n_trials):
        # Only use top 10 neurons
        firing_rates_top10 = result[top_10_neurons, stim_idx, trial_idx]
        preferred_top10 = neurons_preferred[top_10_neurons]

        decoded = decode_orientation(firing_rates_top10, preferred_top10)
        decoded_top10.append(decoded)

print(f"Decoded {len(decoded_top10)} trials using only top 10 neurons")

Decoded 240 trials using only top 10 neurons


In [59]:
# Calculate error for top 10 neurons
df_top10 = pd.DataFrame({
    'stimulus_orientation': np.repeat(stimulus_orientations, n_trials),
    'decoded_orientation': decoded_top10
})

diff_top10 = np.abs(df_top10['stimulus_orientation'] - df_top10['decoded_orientation'])
df_top10['absolute_error'] = np.minimum(diff_top10, 180 - diff_top10)
mean_error_top10 = df_top10['absolute_error'].mean()

print(f"\nComparison:")
print(f"Error with ALL 50 neurons: {mean_error:.2f} degrees")
print(f"Error with TOP 10 neurons: {mean_error_top10:.2f} degrees")
print(f"Difference: {mean_error_top10 - mean_error:.2f} degrees")


Comparison:
Error with ALL 50 neurons: 53.03 degrees
Error with TOP 10 neurons: 42.61 degrees
Difference: -10.42 degrees


### 3.2 Effect of doubling noise

In [60]:
# Function from Task 1 to generate responses
def tuning_curve(stimulus, preferred, r_max=100):
    diff = abs(stimulus - preferred)
    diff = np.minimum(diff, 180 - diff)
    diff_rad = np.radians(diff)
    response = r_max * np.cos(diff_rad)
    return max(0, response)

# Generate new data with 2x noise (multiply Poisson parameter by 2)
result_high_noise = np.zeros((n_neurons, n_stimuli, n_trials))

for i in range(n_neurons):
    for j in range(n_stimuli):
        expected_rate = tuning_curve(stimulus_orientations[j], neurons_preferred[i])
        for k in range(n_trials):
            # Double the Poisson parameter = 2x noise
            result_high_noise[i, j, k] = np.random.poisson(expected_rate * 2)

print("Generated new data with doubled noise")

Generated new data with doubled noise


In [61]:
# Decode with high noise data
decoded_noise = []

for stim_idx in range(n_stimuli):
    for trial_idx in range(n_trials):
        firing_rates_noise = result_high_noise[:, stim_idx, trial_idx]
        decoded = decode_orientation(firing_rates_noise, neurons_preferred)
        decoded_noise.append(decoded)

print(f"Decoded {len(decoded_noise)} trials with doubled noise")

Decoded 240 trials with doubled noise


In [62]:
# Calculate error with doubled noise
df_noise = pd.DataFrame({
    'stimulus_orientation': np.repeat(stimulus_orientations, n_trials),
    'decoded_orientation': decoded_noise
})

diff_noise = np.abs(df_noise['stimulus_orientation'] - df_noise['decoded_orientation'])
df_noise['absolute_error'] = np.minimum(diff_noise, 180 - diff_noise)
mean_error_noise = df_noise['absolute_error'].mean()

print(f"\nComparison:")
print(f"Normal noise: {mean_error:.2f} degrees")
print(f"Doubled noise (2x): {mean_error_noise:.2f} degrees")
print(f"Difference: {mean_error_noise - mean_error:.2f} degrees")


Comparison:
Normal noise: 53.03 degrees
Doubled noise (2x): 4.72 degrees
Difference: -48.32 degrees


## Conclusions

Question 3.1: Top 10 vs All 50 neurons

Using only the top 10 most responsive neurons increased the decoding error compared to using all 50 neurons. This shows that population coding benefits from redundancy, even neurons with lower firing rates contribute useful information. The population vector method works better with more neurons because averaging across a larger population reduces the impact of individual neuron variability.

Question 3.2: Effect of doubling noise

Doubling the noise level (2× Poisson parameter) increased the decoding error slightly. However, the increase was relatively small, demonstrating that population vector decoding is robust to noise when using many neurons. Averaging across 50 neurons helps cancel out random variations, making the decoding more reliable even with higher noise levels.